In [ ]:
%load_ext autoreload
%autoreload 2

import os

# for adding the videos to DB
# don't use at the same time with the server running
# https://stackoverflow.com/questions/59119396/how-to-use-django-3-0-orm-in-a-jupyter-notebook-without-triggering-the-async-con
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from backend.models import Video, UserPreferences, ExpertRating, DjangoUser, UserInformation, EmailDomain
from backend.rating_fields import VIDEO_FIELDS
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML, IFrame, display
from IPython.display import YouTubeVideo
import pandas as pd
from tqdm.auto import tqdm
import gin
from backend.rating_fields import MAX_VALUE
from matplotlib import pyplot as plt
gin.enter_interactive_mode()

In [ ]:
my_username = 'test_my_username'
other_username = 'test_other_username'

# Creating test data

In [ ]:
# creating two certified users
DjangoUser.objects.filter(username__startswith='test_').delete()
dj_my = DjangoUser.objects.create(username=my_username)
dj_other = DjangoUser.objects.create(username=other_username)
up = UserPreferences.objects.create(user=dj_my)
up_other = UserPreferences.objects.create(user=dj_other)
ui = UserInformation.objects.create(user=dj_my)
ui_other = UserInformation.objects.create(user=dj_other)
domain, _ = EmailDomain.objects.get_or_create(domain='@online.com', status=EmailDomain.STATUS_ACCEPTED)
VerifiableEmail.objects.create(user=ui, email='me@online.com', is_verified=True)
VerifiableEmail.objects.create(user=ui_other, email='they@online.com', is_verified=True)
user = up
u = user

In [ ]:
# creating a 1000 videos
Video.objects.filter(video_id__startswith='online').delete()
n_videos = 10
video_ids = ["online-%05d" % i for i in range(n_videos)]
Video.objects.bulk_create([Video(video_id=vid) for vid in video_ids])
video_qs = list(Video.objects.filter(video_id__in=video_ids))
videos_by_id = {video.video_id: video for video in video_qs}
videos = [videos_by_id[vid] for vid in video_ids]

In [ ]:
# creating some random ratings
n_ratings = 10
def get_n_random_pairs(n_videos, n_ratings):
    pairs = set()
    
    def sample_pair():
        v1 = np.random.choice(n_videos - 1) + 1
        v2 = np.random.choice(v1)
        pair = (v1, v2)
        return pair
    
    for i in tqdm(range(n_ratings)):
        pair = sample_pair()
        while pair in pairs:
            pair = sample_pair()
        pairs.add(pair)
    assert all([x > y for x, y in pairs])
    assert len(pairs) == n_ratings
    return list(pairs)

In [ ]:
def create_random_ratings_from_pairs(videos, user, rating_pairs,
                                     nonrandom_r=False):
    to_create = []
    
    for v1, v2 in rating_pairs:
        if np.random.rand() < 0.5:
            (v1, v2) = (v2, v1)
            
        fields = {f: np.random.rand() * MAX_VALUE for f in VIDEO_FIELDS}
        if nonrandom_r and np.random.rand() < 0.05:
            delta = 1. * (v1 - v2) / len(videos) # value: -1, 1
            delta += 1 # value: 0, 2
            delta /= 2 # value: 0, 1
            delta *= MAX_VALUE
            if delta < 0:
                delta = 0
            if delta > MAX_VALUE:
                delta = MAX_VALUE
            fields[VIDEO_FIELDS[0]] = delta

        to_create.append(ExpertRating(user=user, video_1=videos[v1],
                                      video_2=videos[v2],
                                      **fields))
    
    ExpertRating.objects.bulk_create(to_create)

In [ ]:
pairs = get_n_random_pairs(n_videos, n_ratings)
create_random_ratings_from_pairs(videos, user, pairs, nonrandom_r=True)

pairs_other = get_n_random_pairs(n_videos, n_ratings)
create_random_ratings_from_pairs(videos, up_other, pairs_other, nonrandom_r=True)

In [ ]:
# Fitting the model...
from backend.ml_model.client_server.django_ml_featureless import DatabasePreferenceLearnerFeatureless
from django_react.settings import load_gin_config
load_gin_config('../backend/backend/ml_model/config/featureless_config.gin')


# running global fit...
learner = DatabasePreferenceLearnerFeatureless(directory=None,
                                     load=True, save=True)
learner.fit(epochs=1000)
learner.update_features()

# Using test data

In [ ]:
u = UserPreferences.objects.get(user__username=my_username)
n_videos = Video.objects.filter(video_id__startswith='online').count()
video_ids = [x[0] for x in Video.objects.filter(video_id__startswith='online').values_list('video_id')]
video_ids = sorted(video_ids)
videos_qs = Video.objects.filter(video_id__startswith='online')
videos_dct = {video.video_id: video for video in videos_qs}
videos = [videos_dct[video_id] for video_id in video_ids]

In [ ]:
def plot_feature(f):
    plt.xlabel('video id')
    plt.ylabel(f'score [{f}]')
    vrs = VideoRating.objects.filter(user__user__username=my_username)
    feature_map = {int(vr.video.video_id.split('-')[1]): getattr(vr, f) for vr in vrs}
    plt.plot([feature_map.get(i, -1) for i in range(n_videos)], label='my reliability')

    vrs = VideoRating.objects.filter(user__user__username=other_username)
    feature_map = {int(vr.video.video_id.split('-')[1]): getattr(vr, f) for vr in vrs}
    plt.plot([feature_map.get(i, -1) for i in range(n_videos)], label='their reliability')

    vrs = Video.objects.filter(video_id__startswith='online')
    feature_map = {int(vr.video_id.split('-')[1]): getattr(vr, f) for vr in vrs}
    plt.plot([feature_map[i] for i in range(n_videos)], label='total')

    plt.legend()

In [ ]:
plt.figure(figsize=(11, 5))
plt.subplot(1, 2, 1)
plot_feature('reliability')
plt.subplot(1, 2, 2)
plot_feature('backfire_risk')

In [ ]:
videos_by_me = Video.objects.annotate().filter(
    Q(expertrating_video_1__user=u) | Q(expertrating_video_1__user=u)).distinct()

In [ ]:
videos_by_me_and_others = [v for v in videos_by_me if v.rating_n_experts >= 2]

In [ ]:
len(videos_by_me_and_others), len(videos_by_me)

In [ ]:
video_to_change = videos[np.random.choice(n_videos)]

In [ ]:
ratings = ExpertRating.objects.filter(video_1=video_to_change, user=u)

In [ ]:
def rating_other_video(r, v):
    if r.video_1 == v:
        return r.video_2
    elif r.video_2 == v:
        return r.video_1
    else:
        raise Exception()

In [ ]:
random_rating = ratings[np.random.choice(len(ratings))]

In [ ]:
v_other = rating_other_video(random_rating, video_to_change)

In [ ]:
rating = ExpertRating.objects.get(video_1=video_to_change, video_2=v_other,
                                  user=u)

In [ ]:
list(zip(VIDEO_FIELDS, rating.features_as_vector))

In [ ]:
rating.video_1.name, rating.video_2.name

In [ ]:
feature = 'reliability'
# OUR CHANGE

OLD_VAL = getattr(rating, feature)
NEW_VAL = 100

print("CHANGING", rating, feature, OLD_VAL, "to", NEW_VAL)

setattr(rating, feature, 100)

In [ ]:
from backend.ml_model.client_server.django_ml_featureless import DatabasePreferenceLearnerFeatureless
from django_react.settings import load_gin_config

In [ ]:
u_other = UserPreferences.objects.get(user__username=other_username)

In [ ]:
users_to_ratings = {u.id: list(ExpertRating.objects.filter(
    user__user__username=my_username).exclude(id=rating.id).order_by('?')[:10]) + [rating],
                   u_other.id: list(ExpertRating.objects.filter(
    user__user__username=other_username).order_by('?')[:10])}

In [ ]:
def select_adjacent_videos(video, filt, hops=3):
    """Get ratings that are related to the video in k hops."""
    queue = []  # format: (hops, video)
    
    visited = set()
    
    queue.append((0, video.id))
    
    while queue:
        curr_hops, item = queue[0]
        queue = queue[1:]
        
        if item in visited:
            continue
            
        visited.add(item)
        
        curr_res = set()
        related_1 = ExpertRating.objects.filter(filt & Q(video_1__id=item)).values('video_2__id')
        for item_next in related_1:
            curr_res.add(item_next['video_2__id'])
        related_2 = ExpertRating.objects.filter(filt & Q(video_2=item)).values('video_1__id')
        for item_next in related_2:
            curr_res.add(item_next['video_1__id'])
        
        if curr_hops < hops:
            # adding children
            for item_next in curr_res.difference(visited):
                queue.append((curr_hops + 1, item_next))
            
    return visited

In [ ]:
len(select_adjacent_videos(video_to_change, Q(user__id=u.id) | Q(user__id=u_other.id),
   hops=2))

In [ ]:
load_gin_config('../backend/backend/ml_model/config/featureless_config.gin')


In [ ]:
learner = DatabasePreferenceLearnerFeatureless(directory=None,
                                     load=True, save=False,
                                     user_queryset=UserPreferences.objects.filter(Q(id=u.id)|Q(id=u_other.id)),
                                     users_to_ratings=users_to_ratings)

In [ ]:
learner.fit(epochs=1)

# Try GD

In [ ]:
from time import time
losses = []
times = []
t0 = time()

In [ ]:
for _ in tqdm(range(1000)):
    info = learner.aggregator.fit_step()
    losses.append(info['loss'])
    times.append(time() - t0)

In [ ]:
plt.plot(times, losses)

In [ ]:
# Running gradient descent is too slow...

# Running binary search

In [ ]:
mb = learner.aggregator.sample_minibatch(sample_experts=50, sample_ratings_per_expert=10000000,
                                   sample_objects_per_expert=10000)

In [ ]:
learner.aggregator.loss_fcn(**mb)['loss'].numpy()

In [ ]:
mb

In [ ]:
expert_id = learner.aggregator.all_ratings.experts_reverse[u.id]
expert_id_2 = learner.aggregator.all_ratings.experts_reverse[u_other.id]

In [ ]:
feature_id = learner.aggregator.all_ratings.output_features.index(feature)

In [ ]:
object_id = learner.aggregator.all_ratings.objects_reverse[rating.video_1.video_id]

In [ ]:
object_id_2 = learner.aggregator.all_ratings.objects_reverse[rating.video_2.video_id]

In [ ]:
expert_id, object_id, feature_id

In [ ]:
common_expert = learner.aggregator.all_ratings.aggregate_index

In [ ]:
learner.aggregator.all_ratings.layer.v

In [ ]:
learner.aggregator.all_ratings.layer.v[expert_id, object_id, feature_id]

In [ ]:
learner.aggregator.all_ratings.model.layers[1].v

In [ ]:
learner.aggregator.loss_fcn = learner.aggregator.build_loss_fcn(**learner.aggregator.hypers)

In [ ]:
def get_var():
    return learner.aggregator.all_ratings.model.layers[1].v
def set_var(z):
    learner.aggregator.all_ratings.model.layers[1].v = z

In [ ]:
import tensorflow as tf

In [ ]:
mb

In [ ]:
@tf.function
def get_grad_value(expert_id, object_id, feature_id, mb):

#     learner.aggregator.loss_fcn = learner.aggregator.build_loss_fcn(**learner.aggregator.hypers)

    
    with tf.GradientTape() as tape:
        losses = learner.aggregator.loss_fcn(**mb)#, model_tensor=get_var())

    all_variables = learner.aggregator.all_ratings.model.variables
    grads = tape.gradient(losses['loss'], all_variables,
                          unconnected_gradients=tf.UnconnectedGradients.ZERO)
    
    grad_val = grads[0][expert_id, object_id, feature_id]#.numpy()
    return grad_val, losses['loss']#.numpy()


In [ ]:
@tf.function
def set_tensor_val(expert_id, object_id, feature_id, val, var=get_var()):
    var.scatter_nd_update([[expert_id, object_id, feature_id]], [val])

In [ ]:
def get_tensor_val(expert_id, object_id, feature_id):
    return get_var()[expert_id, object_id, feature_id]

In [ ]:
get_tensor_val(expert_id, object_id, feature_id)

In [ ]:
get_grad_value(expert_id, object_id, feature_id, mb)

In [ ]:
l, r = -10, 10

In [ ]:
set_tensor_val(expert_id, object_id, feature_id, l)
get_grad_value(expert_id, object_id, feature_id, mb)

In [ ]:
set_tensor_val(expert_id, object_id, feature_id, r)
get_grad_value(expert_id, object_id, feature_id, mb)

In [ ]:
losses = []
grads = []
theta_vals = []
s_vals = []

theta_vals_2 = []
s_vals_2 = []

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
def binary_search(expert_id, object_id, feature_id, mb):
    i = 0
    l = -10
    r = 10
    while r - l >= 1e-2:
        m = (l + r) / 2
        set_tensor_val(expert_id, object_id, feature_id, m)
        grad, loss = get_grad_value(expert_id, object_id, feature_id, mb)
        if grad < 0:
            l = m
        else:
            r = m

        i += 1
#         print(i, l, r, grad, loss)
#         losses.append(loss)
#         grads.append(grad)
        
#         if object_id == object_id_2:
            
#             theta_vals_2.append(get_tensor_val(0, object_id, feature_id))
#         else:
#             theta_vals.append(get_tensor_val(0, object_id, feature_id))

#         if object_id == object_id_2:
#             s_vals_2.append(get_tensor_val(1, object_id, feature_id))
#         else:
#             s_vals.append(get_tensor_val(1, object_id, feature_id))

In [ ]:
def binary_search_iter(mb):

    binary_search(0, object_id, feature_id, mb)
    binary_search(1, object_id, feature_id, mb)

    binary_search(0, object_id_2, feature_id, mb)
    binary_search(1, object_id_2, feature_id, mb)

In [ ]:
for _ in range(10):
    binary_search_iter(mb)

In [ ]:
%timeit binary_search_iter(mb)

In [ ]:
plt.subplot(1, 3, 1)
plt.plot(losses, label='loss')
plt.plot(np.abs(grads), label='grad')
plt.yscale('log')
plt.legend()
plt.subplot(1, 3, 2)
plt.plot(theta_vals, label='theta')
plt.plot(s_vals, label='s')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(theta_vals_2, label='theta2')
plt.plot(s_vals_2, label='s2')
plt.legend()

In [ ]:
r =[r for r in  learner.user_to_model[u.id].ratings if r['o1'] == rating.video_1.video_id and r['o2'] == rating.video_2.video_id][0]

In [ ]:
def f(val):
    r =[r for r in  learner.user_to_model[u.id].ratings if r['o1'] == rating.video_1.video_id and r['o2'] == rating.video_2.video_id][0]
    
    r['ratings'][feature_id] = val
    
    mb = learner.aggregator.sample_minibatch(sample_experts=50, sample_ratings_per_expert=1000,
                               sample_objects_per_expert=1000)
    
    binary_search_iter(mb)

    theta1 = get_tensor_val(0, object_id, feature_id).numpy()
    s1 = get_tensor_val(1, object_id, feature_id).numpy()
    
    theta2 = get_tensor_val(0, object_id_2, feature_id).numpy()
    s2 = get_tensor_val(1, object_id_2, feature_id).numpy()

    return theta1, s1, theta2, s2

In [ ]:
f(-1)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [ ]:
interact(f, val=widgets.FloatSlider(min=-1, max=1, step=0.05, value=0));


In [ ]:
# fast tensor slice assign

In [ ]:
x = tf.Variable(tf.random_normal())

In [ ]:
variable = tf.Variable(tf.random_uniform_initializer(minval=-1., maxval=1.)(shape=(2, 10000, 8)))

In [ ]:
%timeit variable[0, 1, 2].numpy()

In [ ]:
@tf.function
def get_var_tff():
    return variable[0, 1, 2]

In [ ]:
%timeit get_var_tff()

In [ ]:
def gradient_eager():
    with tf.GradientTape() as tape:
        loss = tf.reduce_mean(tf.sin(tf.abs(variable)))
    return tape.gradient(loss, variable)[0, 1, 2]

In [ ]:
%timeit gradient_eager()

In [ ]:
@tf.function
def gradient_tff():
    loss = tf.reduce_mean(tf.sin(tf.abs(variable)))
    return tf.gradients(loss, [variable])[0][0, 1, 2]

In [ ]:
gradient_tff()

In [ ]:
%timeit gradient_tff()

In [ ]:
var_numpy = variable.numpy()
var_numpy_other = np.copy(var_numpy)
var_numpy_other[0, 1, 2] = 0
mask_one = np.zeros_like(var_numpy)
mask_one[0,  1, 2] = 1.
mask_one_tf = tf.constant(mask_one)
mask_one_tf_bool = tf.constant(mask_one.astype(np.bool))

In [ ]:
var_changing_part = tf.Variable(variable.numpy()[0, 1, 2])

In [ ]:
var_other_part = tf.constant(var_numpy_other)

In [ ]:
%timeit var_changing_part * mask_one_tf + var_other_part - variable

In [ ]:
@tf.function
def fill_rest(changing_part):
    return tf.where(mask_one_tf_bool, var_changing_part, var_other_part)
    #return changing_part * mask_one_tf + var_other_part
    return 

In [ ]:
%timeit tf.where(mask_one_tf_bool, tf.ones_like(var_other_part) * var_changing_part, var_other_part)

In [ ]:
%timeit fill_rest(var_changing_part)

In [ ]:
@tf.function
def gradient_slice_tff():
    loss = tf.reduce_mean(tf.sin(tf.abs(fill_rest(var_changing_part))))
    return tf.gradients(loss, [var_changing_part])[0]

In [ ]:
gradient_slice_tff()

In [ ]:
%timeit gradient_slice_tff()

In [ ]:
%timeit tf.where(mask_one_tf_bool, var_changing_part, var_other_part)

In [ ]:
tf.reduce_max(tf.abs(tf.where(mask_one_tf_bool, var_changing_part, var_other_part) - variable))

In [ ]:
var_changing_part

In [ ]:
@tf.function
def grad_slice_and_fill_rest():
    rec = tf.where(mask_one_tf_bool, var_changing_part, var_other_part)
    loss = tf.reduce_mean(tf.sin(tf.abs(rec)))
    return tf.gradients(loss, [var_changing_part])[0]

In [ ]:
%timeit grad_slice_and_fill_rest()

In [ ]:
def grad_slice_and_fill_rest():
    with tf.GradientTape() as tape:
        rec = tf.where(mask_one_tf_bool, var_changing_part, var_other_part)
        loss = tf.reduce_mean(tf.sin(tf.abs(rec)))
    return tape.gradient(loss, var_changing_part)

In [ ]:
%timeit grad_slice_and_fill_rest()

In [ ]:
variable + tf.sparse.SparseTensor(indices=[[0, 1, 2]],
                       values=[0],
                       dense_shape=variable.shape)

In [ ]:
def slice_assign_dummy()

In [ ]:
tf.tensor_scatter_nd_update(variable, [[0, 1, 2]], [3])

In [ ]:
variable[0, 1, 2]

In [ ]:
%timeit tf.compat.v1.scatter_nd_update(variable, [[0, 1, 2]], [3])

In [ ]:
@tf.function
def scatter_update():
#     return tf.tensor_scatter_nd_update(variable, [[0, 1, 2]], [val])
    tf.compat.v1.scatter_nd_update(variable, [[0, 1, 2]], [51])

In [ ]:
%timeit scatter_update()

In [ ]:
%timeit variable.scatter_nd_update([[0, 1, 2]], [5])

In [ ]:
scatter_update()

In [ ]:
@tf.function
def scatter_var():
    variable.scatter_nd_update([[0, 1, 2]], [5])

In [ ]:
%timeit scatter_var()

In [ ]:
N = np.prod(variable.shape)
lst = np.random.randn(N)

In [ ]:
# WHY IS NUMPY SO MUCH FASTER?????

In [ ]:
%timeit lst[np.random.choice(N)] = 5

In [ ]:
%timeit lst[np.random.choice(N)]

In [ ]:
%timeit tf.gather_nd(variable, [[0, 1, 2]])

In [ ]:
variable

In [ ]:
# trying tf1.0... -- even worse

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
variable = tf.Variable(tf.random_uniform_initializer(minval=-1., maxval=1.)(shape=(2, 10000, 8)))

In [ ]:
session = tf.compat.v1.Session()

In [ ]:
session.run(tf.compat.v1.global_variables_initializer())

In [ ]:
%timeit session.run(variable[0, 1, 2])

In [ ]:
# trying torch...

In [ ]:
import torch
import numpy as np
import tensorflow as tf

In [ ]:
!pip install torch

In [ ]:
var = torch.autograd.Variable(torch.randn(2, 10000, 8), requires_grad=True)

In [ ]:
%timeit var[0, 1, 2]

In [ ]:
%timeit var[0, 1, 2] = np.random.randn()

In [ ]:
def compute_grad():
    var.grad = None
    loss = torch.mean(torch.sin(torch.abs(var)))
    loss.backward()
    return var.grad.data[0, 1, 2].item()

In [ ]:
%timeit compute_grad()

In [ ]:
mask = torch.zeros_like(var, dtype=torch.bool)
mask[0, 1, 2]=True

In [ ]:
var[0, 1, 2] = 5

In [ ]:
var_single = torch.autograd.Variable(torch.from_numpy(np.array(var[0, 1,2].item()).astype(np.float32)), requires_grad=True)

In [ ]:
const_rest = torch.from_numpy(var.detach().numpy().copy()).to(torch.float)
const_rest[0, 1, 2] = 0

In [ ]:
var_single

In [ ]:
var[0, 1, 2]

In [ ]:
const_rest

In [ ]:
const_rest + var_single - var

In [ ]:
%timeit torch.where(mask, var_single, const_rest)

In [ ]:
def compute_grad():
    var_single.grad = None
    var1 = torch.where(mask, var_single, const_rest)
    loss = torch.mean(torch.sin(torch.abs(var1)))
    loss.backward()
    return var_single.grad.data

In [ ]:
%timeit compute_grad()

In [ ]:
# assign is slower on tf but manageable??
# ~800mus anyway spent on gradient computation and loss computation.

In [ ]:
# divide into 3 parts, no grad
def compute_loss():
    loss = torch.mean(torch.sin(torch.abs(var)))
    return loss

In [ ]:
%timeit compute_loss()

In [ ]:
# much faster!

In [ ]:
def variable_index_layer_call_torch(model, idxes1, idxes2):
    return model[idxes1, idxes2]

def loss_fcn_torch(
        experts_rating=None,
        objects_rating_v1=None,
        objects_rating_v2=None,
        cmp=None,
        weights=None,
        experts_all=None,
        objects_all=None,
        num_ratings_all=None,
        objects_common_to_1=None,
        model_tensor=None,
        aggregate_index=None,
        lambda_=None,
        mu=None,
        C=None,
        default_score_value=None,
        **kwargs):
    """
    Compute the loss function. All IDs are internal (int64).

    See https://www.overleaf.com/project/5f44dd8e84c8540001bf1552
    Equations 1-2-3

    Args:
        experts_rating: 1D tensor with expert IDs
        objects_rating_v1: 1D tensor with LEFT objects, same length as experts_rating
        objects_rating_v2: 1D tensor with RIGHT objects, same length as experts_rating
        cmp: 2D tensor comparison_id, feature_id, same length as experts_rating
        weights: 2D tensor comparison_id, feature_weight, same length as experts_rating
        experts_all: 1D tensor with expert IDs for the regularization loss
        objects_all: 1D tensor with objects (for common loss), same length as experts_all
        num_ratings_all: 1D tensor with number of ratings for expert/object in experts_all
            and objects_all, same length as experts_all
        objects_common_to_1: 1D tensor with object IDs for the common-to-1 loss.

    Returns dict of Tensorflow tensors with the total loss and components
    """

    result = {}

    # internal indices for experts and objects (ratings)
#     idx_v1 = torch.stack((experts_rating, objects_rating_v1), dim=1)
#     idx_v2 = torch.stack((experts_rating, objects_rating_v2), dim=1)

#     print(idx_v1)
    
    # 2D array (comparison_id, feature) -> float
    theta_eqv = variable_index_layer_call_torch(model_tensor, experts_rating, objects_rating_v1)
    theta_eqw = variable_index_layer_call_torch(model_tensor, experts_rating, objects_rating_v2)

    # FIT LOSS SUM
    theta_vw = theta_eqv - theta_eqw
    # print(theta_vw.shape, cmp.shape)
    theta_vw_y = torch.mul(theta_vw, cmp)
    sp = torch.nn.Softplus()(theta_vw_y)
    sp_weighted = torch.mul(sp, weights)

    sp_weighted_flat = sp_weighted.view((-1,))
    sp_weighted_no_nan = sp_weighted_flat[~torch.isnan(sp_weighted_flat)]
    # tf.print("original tensor")
    # tf.print(sp_weighted_flat)

    # tf.print("nonan tensor")
    # tf.print(sp_weighted_no_nan)

    result['loss_fit'] = torch.sum(sp_weighted_no_nan)

    # LOSS MODEL TO COMMON
    # common expert
    experts_common = torch.full(experts_all.shape, aggregate_index, dtype=torch.int64)

    # indices for experts for regularization
#     idx_all = torch.stack((experts_all, objects_all), dim=1)
#     idx_common = torch.stack((experts_common, objects_all), dim=1)

    # 2D array (regul_id, feature) -> float
    theta_eqv_common = variable_index_layer_call_torch(model_tensor, experts_all, objects_all)
    s_qv = variable_index_layer_call_torch(model_tensor, experts_common, objects_all)

    # print("IDX", idx_common.shape, s_qv.shape)

    # coefficient, shape: regul_id
    num_float = num_ratings_all.to(torch.float32)
    coef_yev = torch.div(num_float, C + num_float)
    # print(theta_eqv_common.shape, s_qv.shape)
    # tf.print('thetacomm', theta_eqv_common)
    # tf.print("sqv", s_qv)
    theta_s = torch.abs(theta_eqv_common - s_qv)
    coef_yev_repeated = torch.unsqueeze(
            coef_yev,
            dim=1).repeat(
        1,
        theta_s.shape[1])
    # print("THETAS", theta_s.shape, "COEF", coef_yev.shape, \
    # "COEFR", coef_yev_repeated.shape)
    # tf.print("thetas", theta_s)
    # tf.print("coefyev", coef_yev_repeated)
    theta_s_withcoeff = torch.mul(theta_s, coef_yev_repeated)
    result['loss_m_to_common'] = torch.sum(
        theta_s_withcoeff) * lambda_

    
    
    # LOSS COMMON TO 0
    experts_common_to_1 = torch.full(objects_common_to_1.shape,
                                     aggregate_index, dtype=torch.int64)
#     idx_common_to_1 = torch.stack(
#         (experts_common_to_1, objects_common_to_1), dim=1)
    s_qv_common_to_1 = variable_index_layer_call_torch(model_tensor, experts_common_to_1, objects_common_to_1)

    sm1 = torch.pow(s_qv_common_to_1 - default_score_value, 2.0)

    # print(idx_common_to_1, s_qv_common_to_1, sm1)

    result['loss_common_to_1'] = torch.sum(sm1) * mu

    # TOTAL LOSS
    result['loss'] = result['loss_fit'] + result['loss_m_to_common'] + result[
        'loss_common_to_1']

    return result

In [ ]:
mb_np = {x: y.numpy() for x, y in mb.items()}

In [ ]:
np_model = var.numpy()

In [ ]:
loss_fcn = loss_fcn_np(**mb_np, **learner.aggregator.hypers, model_tensor=np_model)['loss']

In [ ]:
%timeit loss_fcn_np(**mb_np, **learner.aggregator.hypers, model_tensor=np_model)['loss']

In [ ]:
# -++ # delete rightmost
# --+ # delete leftmost
# +-- # delete leftmost
# ++- # delete rightmost

In [ ]:
def trisection(fcn, l, r, get_var, set_var, eps=1e-3):
    def get_m1_m2(l, r):
        delta = (r - l) / 3.
        m1 = l + delta
        m2 = l + 2 * delta
        return m1, m2
    while r - l > eps:
        m1, m2 = get_m1_m2(l, r):
        

In [ ]:
from scipy.optimize import golden

In [ ]:
?golden

In [ ]:
def func(x):
    np_model[1, 5, 6] = x
    return loss_fcn_np(**mb_np, **learner.aggregator.hypers, model_tensor=np_model)['loss']

In [ ]:
func(11)

In [ ]:
%timeit golden(func, full_output=1)

In [ ]:
?golden

In [ ]:
golden(func, full_output=1)

In [ ]:
from backend.ml_model.preference_aggregation_featureless_online import FeaturelessOnlineUpdater

In [ ]:
#%timeit learner.aggregator.loss_fcn(**mb)['loss'].numpy()


In [ ]:
mb_torch = {x: torch.from_numpy(y.numpy()) for x, y in mb.items()}

In [ ]:
learner.aggregator.hypers

In [ ]:
torch_model = torch.autograd.Variable(torch.from_numpy(learner.aggregator.all_ratings.layer.v.numpy()),
                                      requires_grad=True)

In [ ]:
loss_fcn_torch(**mb_torch, **learner.aggregator.hypers, model_tensor=torch_model)

In [ ]:
learner.aggregator.loss_fcn(**mb)

In [ ]:
%timeit learner.aggregator.loss_fcn(**mb)

In [ ]:
%timeit loss_fcn_torch(**mb_torch, **learner.aggregator.hypers, model_tensor=torch_model)

In [ ]:
var = learner.aggregator.all_ratings.model.variables[0]

In [ ]:
loss_fn = learner.aggregator.build_loss_fcn(**learner.aggregator.hypers)

In [ ]:
@tf.function
def grad_compute_tf():
#     del mb['']
    loss = loss_fn(**mb)['loss']
    grad = tf.gradients(loss, [var])[0]
    return grad[1, 2, 3]

In [ ]:
%timeit grad_compute_tf()

In [ ]:
grad_compute_tf()

In [ ]:
def grad_compute_torch():
    torch_model.grad = None
    loss = loss_fcn_torch(**mb_torch, **learner.aggregator.hypers, model_tensor=torch_model)['loss']
    loss.backward()
    return torch_model.grad.data[1, 2, 3]

In [ ]:
%timeit grad_compute_torch()

In [ ]:
var = learner.aggregator.all_ratings.layer.v

In [ ]:
var[0, 1, 2]

In [ ]:
%timeit var.numpy()

In [ ]:
np_var = var.numpy()

In [ ]:
np_var[0, 1, 2] = 100

In [ ]:
var[0, 1, 2]

In [ ]:
var_const = tf.constant(var)

In [ ]:
nparr = np.asarray(memoryview(var_const))
nparr.setflags(write=1)

In [ ]:
var_const[0, 1, 2] = 10

In [ ]:
nparr[0, 1, 2] = 10

In [ ]:
var[0, 1, 2]

In [ ]:
nparr[0, 1, 2]

In [ ]:
np.prod(np_model.shape)

In [ ]:
1000*5000*10

In [ ]:
2*1000*10

In [ ]:
learner.aggregator.hypers

In [ ]:
gin.query_parameter('FeaturelessMedianPreferenceAverageRegularizationAggregator.hypers')

# Using Updater class

In [ ]:
from django_react.settings import load_gin_config
load_gin_config('../backend/backend/ml_model/config/featureless_config.gin')


In [ ]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

from backend.ml_model.preference_aggregation_featureless_online import FeaturelessOnlineUpdater

#### FILL DATA FROM LEARNER

In [ ]:
online = FeaturelessOnlineUpdater(golden_params={'maxiter': 20, 'tol': 1e-8, 'smartbracket': (-1, 1)})

In [ ]:
online = FeaturelessOnlineUpdater(golden_params={'maxiter': 20, 'tol': 1e-8, 'smartbracket': (-1, 1)})
online.hypers['aggregate_index'] = common_expert


In [ ]:
logging.info('sdf')

In [ ]:
online.set_minibatch({x: y.numpy() for x, y in mb.items()})

In [ ]:
online.set_model_tensor(learner.aggregator.all_ratings.model.variables[0].numpy())

In [ ]:
online.set_subtract()

In [ ]:
indices_lst = [
    (expert_id,     object_id,   feature_id),
    (expert_id,     object_id_2, feature_id),
    
    (expert_id_2,   object_id,   feature_id),
    (expert_id_2,   object_id_2, feature_id),
    
    (common_expert, object_id,   feature_id),
    (common_expert, object_id_2, feature_id),
] * 1

In [ ]:
initial_value = {ind: online.get_closure_loss(ind)(online.get_value(ind)) for ind in set(indices_lst)}

In [ ]:
online.get_value(ind)

In [ ]:
result = online.best_value_many_indices(indices_lst, assign_at_end=True)

In [ ]:
%timeit result = online.best_value_many_indices(indices_lst)

In [ ]:
usernames = [my_username, other_username]

In [ ]:
# usernames to update
ups = get_from_list(UserPreferences.objects.all(), 'user__username', usernames)

In [ ]:
# obtaining video to rate
video1 = VideoRating.objects.filter(user__user__username=my_username).order_by('?')[0].video

In [ ]:
video2 = ExpertRating.objects.filter(user__user__username=my_username, video_1=video1).order_by('?')[0].video_2

### FILL DATA FROM DB

In [ ]:
online = FeaturelessOnlineUpdater()

In [ ]:
user_set_value = 'test_other_username'

In [ ]:
rating_to_change = ExpertRating.objects.filter(user__user__username=user_set_value).order_by('?')[0]
video1 = rating_to_change.video_1
video2 = rating_to_change.video_2
field_to_change = VIDEO_FIELDS[0]

In [ ]:
update_context = OnlineRatingUpdateContext(rating_to_change, field_to_change)

In [ ]:
from copy import deepcopy

In [ ]:
mb_np_orig = deepcopy(mb_np)
model_tensor_orig = deepcopy(model_tensor)

In [ ]:
update_context.model_tensor

In [ ]:
gin.bind_parameter('FeaturelessOnlineUpdater.golden_params',
                  {'maxiter': 20, 'tol': 1e-8, 'smartbracket': (-1, 1)})

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
from functools import partial
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
print("### Online updates demo")

print("Changing", field_to_change, "on", video1, '/', video2)
print("Rating by", user_set_value, "showing rating by", usernames[user_get_value])
print("Other users", len(usernames) - 1)
print("Ratings in the loss", len(ratings_selected))
print("Videos in the loss", len(videos_selected))


pbar = IntProgress(min=0, max=100)
display(pbar) # display the bar

fcn = partial(compute_online_update, pbar=pbar)
fcn.__name__ = "Online updates"
interact(fcn, rating_value=widgets.FloatSlider(min=-1, max=1, step=0.05, value=0),
         pbar=fixed(pbar), mb_np_orig=fixed(mb_np_orig), model_tensor_orig=fixed(model_tensor_orig),
         idx_set=fixed(idx_set), user_get_value=fixed(-1),#,fixed(user_get_value),
         maxiter=20, n_repeats=1, tol=fixed(1e-8),
         hotfix_update_hypers=fixed({'mu': 1, 'lambda_': 0.1}))